In [1]:

#Imports

import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
from os import listdir
import os
import time
import gc
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold

from src3 import *

In [2]:
import sklearn
print(sklearn.__version__)


0.20.3


In [3]:
directory = r"C:\Users\BERK\Dropbox\Time Series Classification\Vary"
datasets = listdir(directory)

In [4]:
datasets[0]

'AllGestureWiimoteX'

In [5]:
datasets.index('Adiac')

0

In [15]:
np.array(temp_list_names)[[0,1,2,3,6]].tolist()

['ChlorineConcentration',
 'HandOutlines',
 'ECG5000',
 'StarLightCurves',
 'UWaveGestureLibraryZ']

In [17]:
datasets[13]

'PLAID'

In [4]:
test_results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','test_acc','param_time','train_time','test_time'])
results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','oob_acc'])

depth_cv = [10]
ntree_cv = [100]
term_cv = [True]
rep_num = 1
method_list = ['b']
distance_list = ['manhattan']
normal_list = [False]
batch_list = list(range(10))
batch_list2 = list(range(100))
data_name="CBF"
directory = r"C:\Users\BERK\Dropbox\Time Series Classification\Vary\\"
datasets = listdir(directory)
#datasets.sort()
#dsum = data_summary(directory)

#run_data = dsum.sort_values(by=['total'])['data'].values[60:70].tolist()

for ddd in [datasets[0]]:
	data_name = ddd
	print(data_name)
	train,test,labels_train,labels_test,X_train,X_test = load_data_new(directory,data_name)
	#print(data_name+" is loaded!")
	results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','var','oob_acc'])
	param_time = time.time()
	
	for method in method_list:

		represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
		#print(data_name+" is prepared!")
		#print("CV starts")
		for distance_measure in distance_list:
			for rep in range(rep_num):
				print(rep)
				for depth in depth_cv:
					for ntree in ntree_cv:
						for is_terminal in term_cv: 
							for n in [0]:
								first = True
								counter = 0
								for b in batch_list:
									represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
									represent_train['Id'] = train_id
									represent_train = represent_train.dropna()
									#represent_train = represent_train.fillna(-1000)
									train_id = represent_train['Id'].values
									represent_train = represent_train.drop(columns = ['Id'])


									represent_test['Id'] = test_id
									represent_test = represent_test.dropna()
									#represent_test = represent_test.fillna(-1000)
									test_id = represent_test['Id'].values
									represent_test = represent_test.drop(columns = ['Id'])
                                    
									trainbow,testbow = learn_representation_sparse_2(represent_train, None,labels_train,labels_test, train_id, test_id, depth, int(ntree/len(batch_list)),rep*1000+counter, is_terminal,normal=False)
									counter = counter +1
									#sel = VarianceThreshold(threshold=0)
									#trainbow = sel.fit_transform(trainbow)
									#testbow = sel.fit_transform(testbow)                                    
									#valss = np.percentile(np.var(trainbow.todense(),axis=0),n,axis=1)
                                    
									#sel = VarianceThreshold(threshold=valss)
									#trainbow = sel.fit_transform(trainbow)
									#testbow = sel.fit_transform(testbow)                       
                                    
									if first is True:
										distTrain=pairwise_distances(trainbow,metric="manhattan",n_jobs=-1)       
										first = False
									else:
										distTrain=distTrain+ pairwise_distances(trainbow,metric="manhattan",n_jobs=-1)
									del trainbow
									del testbow
									gc.collect()
								np.fill_diagonal(distTrain, 1000000000000)
								nncl=KNeighborsClassifier(n_neighbors=1,metric='precomputed')
								nnfit=nncl.fit(distTrain,labels_train)
								predicted_train = nnfit.predict(distTrain)
								cv_accuracy = accuracy_score(labels_train,predicted_train)
								#test_accuracy = test_model(trainbow,testbow,labels_train,labels_test,metric = distance_measure) 
								temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[False],'rep':[rep+1],'depth':[depth],'ntree':[ntree],'var':[n],'type':[is_terminal],'oob_acc':[cv_accuracy]})

                                #temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[n],'rep':[rep+1],'depth':[depth],'ntree':[ntree],'type':[is_terminal],'oob_acc':[cv_accuracy],'test_acc':[test_accuracy]})
								#print(temp_results)
								results = results.append(temp_results,sort=False)
							
                                
	
	summary = results.groupby(['method','normalize','depth','var']).mean().reset_index()[results.groupby(['method','normalize','depth','var']).mean().reset_index()['oob_acc'] == results.groupby(['method','normalize','depth','var']).mean().reset_index()['oob_acc'].max()].reset_index(drop=True).sort_values(['method'],ascending=False).iloc[0]
	#summary = {'method':'d','normalize':False,'depth':10,'var':25}
	method = summary['method']
	n = summary['normalize']
	depth = summary['depth']
	var = summary['var']
	print('Parameter selection is completed')
	param_time = time.time()-param_time

	for rep in range(20):
		train_time = time.time()
		first = True
		counter = 0
		for b in batch_list2:
			
			represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
            
			represent_train['Id'] = train_id
			represent_train = represent_train.dropna()
			#represent_train = represent_train.fillna(-1000)

			train_id = represent_train['Id'].values
			represent_train = represent_train.drop(columns = ['Id'])


			represent_test['Id'] = test_id
			represent_test = represent_test.dropna()
			#represent_test = represent_test.fillna(-1000)
			test_id = represent_test['Id'].values
			represent_test = represent_test.drop(columns = ['Id'])
            
			trainbow,testbow = learn_representation_sparse_2(represent_train, represent_test,labels_train,labels_test, train_id, test_id, depth, int(500/len(batch_list2)),rep*1000+counter, is_terminal,normal=n)
			counter = counter +1
			#print(rep+counter)
			sel = VarianceThreshold(threshold=0)
			trainbow = sel.fit_transform(trainbow)
			testbow = sel.transform(testbow)                                    
									
			#valss = np.percentile(np.var(trainbow.todense(),axis=0),var,axis=1)
                                    
			#sel = VarianceThreshold(threshold=valss)
			trainbow_1 = trainbow
			testbow_1 = testbow   
            

			if first is True:
				distTrainTest=pairwise_distances(trainbow_1,testbow_1,metric='manhattan',n_jobs=-1) 
				first = False
			else:
				distTrainTest = distTrainTest +pairwise_distances(trainbow_1,testbow_1,metric='manhattan',n_jobs = -1) 
			del trainbow
			del testbow
			gc.collect()
        
		train_time = time.time()-train_time
		test_time = time.time()
		predicted_test = np.array(labels_train)[np.argmin(distTrainTest, axis=0)]
		test_accuracy=accuracy_score(labels_test,predicted_test)
  
		test_time = time.time()-test_time
		temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[n],'rep':[rep+1],'depth':[depth],'ntree':[400],'type':[is_terminal],'test_acc':[test_accuracy],'param_time':[np.round(param_time,1)],'train_time':[np.round(train_time,1)],'test_time':[np.round(test_time,1)]})
		test_results = test_results.append(temp_results,sort=False)

		print(test_accuracy)    
	test_results.to_csv('AllResultsNewApp_Vary_Full_3.csv')
print(np.mean(test_results['test_acc']))

AllGestureWiimoteX
0
Parameter selection is completed
0.6
0.6157142857142858
0.6028571428571429
0.5985714285714285


KeyboardInterrupt: 

In [14]:
datasets[13]

'PLAID'

In [6]:
test_results.groupby('data_name').mean()

,test_acc,param_time,train_time,test_time
data_name,,,,
Adiac,0.787468,2337.5,73.960,0.0
ArrowHead,0.818571,657.3,68.090,0.0
Beef,0.800000,664.6,67.090,0.0
BeetleFly,0.705000,657.3,66.320,0.0
BirdChicken,0.862500,656.5,66.205,0.0
...,...,...,...,...
ShapeletSim,0.912222,676.7,70.085,0.0
ShapesAll,0.857917,5075.2,112.870,0.0
SmallKitchenAppliances,0.686800,2597.5,103.220,0.0


In [6]:
p1 = pd.read_csv('AllResultsNewApp_FS_Simple_Best_Additional_All_New_1.csv')
p2 = pd.read_csv('AllResultsNewApp_FS_Simple_Best_Additional_All_New_2.csv')
p3 = pd.read_csv('AllResultsNewApp_FS_Simple_Best_Additional_All_New_3.csv')
p4 = pd.read_csv('AllResultsNewApp_FS_Simple_Best_Additional_All_New_4.csv')
p5 = pd.read_csv('AllResultsNewApp_FS_Simple_Best_Additional_All_New_5.csv')
p6 = pd.read_csv('AllResultsNewApp_FS_Simple_Best_Additional_All_New_6.csv')
p7 = pd.read_csv('AllResultsNewApp_FS_Simple_Best_Additional_All_New_7.csv')
p8 = pd.read_csv('AllResultsNewApp_FS_Simple_Best_Additional_All_New_8.csv')

In [7]:
temp_list_names=list(set(datasets).difference(pd.concat([p1,p2,p3,p4,p5,p6,p7,p8])['data_name'].unique().tolist()))


In [9]:
p8

,Unnamed: 0,data_name,method,distance_measure,normalize,rep,depth,ntree,test_acc,param_time,train_time,test_time,type
0,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,1,12,400,0.824742,4183.9,75.8,0.0,True
1,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,2,12,400,0.838488,4183.9,75.9,0.0,True
2,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,3,12,400,0.824742,4183.9,75.9,0.0,True
3,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,4,12,400,0.831615,4183.9,75.8,0.0,True
4,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,5,12,400,0.828179,4183.9,76.1,0.0,True
5,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,6,12,400,0.838488,4183.9,75.7,0.0,True
6,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,7,12,400,0.838488,4183.9,76.1,0.0,True
7,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,8,12,400,0.831615,4183.9,75.9,0.0,True
8,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,9,12,400,0.835052,4183.9,76.0,0.0,True
9,0,ProximalPhalanxOutlineCorrect,d,manhattan,False,10,12,400,0.828179,4183.9,76.0,0.0,True


In [13]:
np.array(temp_list_names)[[0,1,2,3,6]]

array(['ChlorineConcentration', 'HandOutlines', 'ECG5000',
       'StarLightCurves', 'UWaveGestureLibraryZ'], dtype='<U22')

In [7]:
test_results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','test_acc','param_time','train_time','test_time'])
results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','oob_acc'])

depth_cv = [6,8,10,12]
ntree_cv = [100]
term_cv = [True]
rep_num = 10
method_list = ['l','d','b']
distance_list = ['manhattan']
normal_list = [False]
batch_list = list(range(10))

data_name="CBF"
directory = "/home/ml_berkgorgulu/Desktop/TSCProblems/"
datasets = listdir(directory)
datasets.sort()
#dsum = data_summary(directory)

#run_data = dsum.sort_values(by=['total'])['data'].values[60:70].tolist()

for ddd in [65]:
	data_name = datasets[ddd]
	print(data_name)
	train,test,labels_train,labels_test,X_train,X_test = load_data(directory,data_name)
	#print(data_name+" is loaded!")
	results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','var','oob_acc'])
	param_time = time.time()
	for method in method_list:
		represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
		#print(data_name+" is prepared!")
		#print("CV starts")
		for distance_measure in distance_list:
			for rep in range(rep_num):
				print(rep)
				for depth in depth_cv:
					for ntree in ntree_cv:
						for is_terminal in term_cv: 
							for n in [0,10,25,50,75]:
								first = True
								for b in batch_list:
									represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)

									trainbow,testbow = learn_representation_sparse_2(represent_train, None,labels_train,labels_test, train_id, test_id, depth, int(ntree/len(batch_list)),rep, is_terminal,normal=False)
									sel = VarianceThreshold(threshold=0)
									trainbow = sel.fit_transform(trainbow)
									#testbow = sel.fit_transform(testbow)                                    
									valss = np.percentile(np.var(trainbow.todense(),axis=0),n,axis=1)
                                    
									sel = VarianceThreshold(threshold=valss)
									trainbow = sel.fit_transform(trainbow)
									#testbow = sel.fit_transform(testbow)                       
                                    
									if first is True:
										distTrain=pairwise_distances(trainbow,metric="manhattan",n_jobs=-1)       
										first = False
									else:
										distTrain=distTrain+ pairwise_distances(trainbow,metric="manhattan",n_jobs=-1)
									del trainbow
									del testbow
									gc.collect()
								np.fill_diagonal(distTrain, 1000000000000)
								nncl=KNeighborsClassifier(n_neighbors=1,metric='precomputed')
								nnfit=nncl.fit(distTrain,labels_train)
								predicted_train = nnfit.predict(distTrain)
								cv_accuracy = accuracy_score(labels_train,predicted_train)
								#test_accuracy = test_model(trainbow,testbow,labels_train,labels_test,metric = distance_measure) 
								temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[False],'rep':[rep+1],'depth':[depth],'ntree':[ntree],'var':[n],'type':[is_terminal],'oob_acc':[cv_accuracy]})

                                #temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[n],'rep':[rep+1],'depth':[depth],'ntree':[ntree],'type':[is_terminal],'oob_acc':[cv_accuracy],'test_acc':[test_accuracy]})
								#print(temp_results)
								results = results.append(temp_results,sort=False)

                                
	
	#summary = results.groupby(['method','normalize','depth','var']).mean().reset_index()[results.groupby(['method','normalize','depth','var']).mean().reset_index()['oob_acc'] == results.groupby(['method','normalize','depth','var']).mean().reset_index()['oob_acc'].max()].reset_index(drop=True).sort_values(['method'],ascending=False).iloc[0]
	summary = {'method':'d','normalize':False,'depth':10,'var':10}
	method = summary['method']
	n = summary['normalize']
	depth = summary['depth']
	var = summary['var']
	is_terminal = True
	print('Parameter selection is completed')
	param_time = time.time()-param_time
	for rep in range(20):
		train_time = time.time()
		first = True
		batch_list = list(range(500))
		for b in batch_list:
			represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
			trainbow,testbow = learn_representation_sparse_2(represent_train, represent_test,labels_train,labels_test, train_id, test_id, depth, int(500/len(batch_list)),rep, is_terminal,normal=n)
			

			sel = VarianceThreshold(threshold=0)
			trainbow = sel.fit_transform(trainbow)
			testbow = sel.transform(testbow)                                    
									
			valss = np.percentile(np.var(trainbow.todense(),axis=0),var,axis=1)
                                    
			sel = VarianceThreshold(threshold=valss)
			trainbow_1 = sel.fit_transform(trainbow)
			testbow_1 = sel.transform(testbow)     
            

			if first is True:
				distTrainTest=pairwise_distances(trainbow_1,testbow_1,metric='manhattan',n_jobs=-1) 
				first = False
			else:
				distTrainTest = distTrainTest +pairwise_distances(trainbow_1,testbow_1,metric='manhattan',n_jobs = -1) 
			del trainbow
			del testbow
			gc.collect()
        
		train_time = time.time()-train_time
		test_time = time.time()
		predicted_test = np.array(labels_train)[np.argmin(distTrainTest, axis=0)]
		test_accuracy=accuracy_score(labels_test,predicted_test)
		distance_measure = 'manhattan'
		test_time = time.time()-test_time
		temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[n],'rep':[rep+1],'depth':[depth],'ntree':[400],'type':[is_terminal],'test_acc':[test_accuracy],'param_time':[np.round(param_time,1)],'train_time':[np.round(train_time,1)],'test_time':[np.round(test_time,1)]})
		test_results = test_results.append(temp_results,sort=False)

		print(test_accuracy)    
	test_results.to_csv('AllResultsNewApp_FS_Simple_Best_Additional_3.csv')
print(np.mean(test_results['test_acc']))

StarLightCurves
Parameter selection is completed
0.8845313258863526
0.9582321515298688
0.8697183098591549
0.9465760077707625
0.9061437591063624
0.9199854298203011
0.9052938319572608
0.8307430791646431


KeyboardInterrupt: 

In [37]:
test_results.groupby('data_name').median()

,test_acc,param_time,train_time,test_time
data_name,,,,
ElectricDevices,0.680781,571668.1,459.65,0.1
FordA,0.713135,38877.7,2286.45,0.0
FordB,0.691144,18079.0,1362.30,0.0
NonInvasiveFatalECGThorax1,0.852417,55808.8,2618.60,0.0
NonInvasiveFatalECGThorax2,0.881934,50825.3,1044.95,0.0
PhalangesOutlinesCorrect,0.756993,14679.0,188.25,0.0
Phoneme,0.241561,7023.1,360.60,0.0
ShapesAll,0.855000,12883.1,161.10,0.0


In [39]:
selected_summary = summary

In [ ]:
test_results.to_csv('AllResultsNewApp_FS_Simple_Best_11.csv')

In [ ]:
test_results.to_csv('AllResultsNewApp_FS_Simple_Best_9.csv')

In [ ]:
path = directory+data_name+"\\"
np.genfromtxt(path+data_name+'_TRAIN.txt')

In [ ]:
path+data_name

In [ ]:
sel = VarianceThreshold(threshold=0)
t2 = sel.fit_transform(trainbow)

In [ ]:
path = directory+data_name+"\\"
train = np.genfromtxt(path+data_name+'_TRAIN.txt')
test = np.genfromtxt(path+data_name+'_TEST.txt')

labels_train= np.array(train[:,0],dtype=np.int16)
X_train= train[:,1:train.shape[1]]
labels_test= np.array(test[:,0],dtype=np.int16)
X_test= test[:,1:test.shape[1]]

In [ ]:
X_train

In [ ]:
represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
trainbow,testbow = learn_representation_sparse_2(represent_train, represent_test,labels_train,labels_test, train_id, test_id, 10, 500,rep, is_terminal,normal=n)
distTrainTest=pairwise_distances(trainbow,testbow,metric='manhattan') 
predicted_test = np.array(labels_train)[np.argmin(distTrainTest, axis=0)]
accuracy_score(labels_test,predicted_test)

In [ ]:
distTrainTest=pairwise_distances(trainbow[:,reg.coef_!=0],testbow[:,reg.coef_!=0],metric='manhattan') 
predicted_test = np.array(labels_train)[np.argmin(distTrainTest, axis=0)]
accuracy_score(labels_test,predicted_test)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
log = LogisticRegressionCV(Cs=10)
log.fit(trainbow, labels_train)
accuracy_score(labels_test,log.predict(testbow))

In [ ]:
trainbow

In [ ]:
distTrainTest2=pairwise_distances(trainbow[:,(log.coef_!=0)[0]],testbow[:,(log.coef_!=0)[0]],metric='manhattan') 
predicted_test = np.array(labels_train)[np.argmin(distTrainTest2, axis=0)]
accuracy_score(labels_test,predicted_test)

In [ ]:
np.array_equal(distTrainTest,distTrainTest2)

In [ ]:
trainbow[:,(log.coef_!=0)[0]]

In [ ]:
trainbow

In [ ]:
np.sort(test_results['test_acc'])

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=100)
clf = clf.fit(trainbow, labels_train)
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)

trainbow_1 = model.transform(trainbow)
testbow_1 = model.transform(testbow)

distTrainTest2=pairwise_distances(trainbow_1,testbow_1,metric='manhattan') 
predicted_test = np.array(labels_train)[np.argmin(distTrainTest2, axis=0)]
accuracy_score(labels_test,predicted_test)

In [ ]:
clf = RandomForestClassifier(n_estimators=50)
clf = clf.fit(trainbow, labels_train)
accuracy_score(labels_test,clf.predict(testbow))

In [ ]:
trainbow_1.shape

In [ ]:
summary

In [ ]:
train,test,labels_train,labels_test,X_train,X_test = load_data(directory,'UWaveGestureLibraryX')
represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode='d')

trainbow,testbow = learn_representation(represent_train, represent_test,labels_train,labels_test, train_id, test_id, depth, ntree,rep, is_terminal,normal=n)


In [ ]:
trainbow

In [ ]:
rt = RandomTreesEmbedding(max_depth=depth,n_estimators=ntree,random_state =1)

traincv=represent_train
testcv=represent_test
trainind=np.unique(train_id)
testind=np.unique(test_id)

trainlabels=labels_train
testlabels=labels_test

randTrees=rt.fit(traincv.values)


In [ ]:
temp = pd.DataFrame(trainRep)
temp['ID'] = train_id 

In [ ]:
trainRep[:,]
train_id

In [ ]:
def train_f(c):
    global train_id
    return sparse.coo_matrix(([1]*len(train_id), (train_id, c)))

In [ ]:
k = time.time()
trainRep=randTrees.apply(traincv.values)
testRep=randTrees.apply(testcv.values)
a = np.apply_along_axis(train_f, 0, trainRep)
sparse.hstack(a)
print(time.time()-k)

In [ ]:
k = time.time()
trainRep=randTrees.transform(traincv.values)
testRep=randTrees.transform(testcv.values)

newId=np.unique(train_id)
Mask = sparse.csr_matrix((np.ones(traincv.shape[0],int),(train_id, np.arange(traincv.shape[0]))), shape=(newId.shape[0],traincv.shape[0]))

trainbow = Mask * trainRep

newId=np.unique(test_id)
Mask = sparse.csr_matrix((np.ones(testcv.shape[0],int),(test_id, np.arange(testcv.shape[0]))), shape=(newId.shape[0],testcv.shape[0]))

testbow = Mask * testRep              

print(time.time()-k)

In [ ]:
pd.Categorical.from_array(trainRep)

In [ ]:
pd.DataFrame(trainRep).apply(pd.value_counts)

In [ ]:
temp2 = pd.get_dummies(pd.DataFrame(trainRep).astype('category'),sparse=True)
temp2['ID'] = train_id
temp2.groupby('ID').sum()

In [ ]:
pd.crosstab(train_id,np.unique(trainRep),trainRep,aggfunc='count')

In [ ]:
np.unique(trainRep)

In [ ]:
pd.get_dummies(pd.DataFrame(trainRep))

In [ ]:
pd.crosstab(train_id,np.unique(trainRep))

In [ ]:
pd.pivot_table()

In [ ]:
trainRep=randTrees.apply(traincv.astype(np.float32).values)



In [ ]:
temp = pd.DataFrame(trainRep)
temp['ID'] = train_id

In [ ]:
t

In [ ]:
temp.groupby('ID').size().unstack(fill_value=0)

In [ ]:
pd.DataFrame(trainRep).apply(lambda x: x.value_counts())

In [ ]:
newId=np.unique(train_id)
Mask = sparse.csr_matrix((np.ones(traincv.shape[0],int),(train_id, np.arange(traincv.shape[0]))), shape=(newId.shape[0],traincv.shape[0]))

trainbow = Mask * trainRep

newId=np.unique(test_id)
Mask = sparse.csr_matrix((np.ones(testcv.shape[0],int),(test_id, np.arange(testcv.shape[0]))), shape=(newId.shape[0],testcv.shape[0]))

testbow = Mask * testRep              

if normal:
    trainbow = normalize(trainbow, norm='l1', axis=1)
    testbow = normalize(testbow, norm='l1', axis=1)


In [ ]:
represent_train.memory_usage().sum()/1000000

In [ ]:
test_results = pd.DataFrame(columns=['data_name','method','distance_measure','normalize','rep','depth','ntree','test_acc'])

for rep in range(10):
	represent_train, represent_test, train_id, test_id = prepare_data(X_train,X_test,mode=method)
	trainbow,testbow = learn_representation(represent_train, represent_test,labels_train,labels_test, train_id, test_id, 8, 500, is_terminal,normal=n)
	test_accuracy = test_model(trainbow,testbow,labels_train,labels_test,metric = 'euclidean')  
	temp_results = pd.DataFrame({'data_name':[data_name],'method':method,'distance_measure':[distance_measure],'normalize':[n],'rep':[rep+1],'depth':[depth],'ntree':[200],'type':[is_terminal],'test_acc':[test_accuracy]})
	test_results = test_results.append(temp_results,sort=False)
print(test_results)

In [ ]:
test_results['test_acc'].mean()

In [ ]:
results.groupby(['method','normalize','depth']).mean()

In [ ]:
np.round(time.time()-a,0)

In [ ]:
from sklearn.preprocessing import normalize

In [11]:
np.median([0.688237582674102,
0.6925171832447153,
0.6965374140837765,
0.6932952924393724,
0.6932952924393724,
0.7028919725068085,
0.69303592270782,
0.6903125405265206,
0.6897938010634159,
0.6939437167682532,
0.6960186746206718,
0.6931656075735961,
0.700946699520166,
0.6961483594864479,
0.6916093891842822,
0.6958889897548955,
0.6909609648554014,
0.6943327713655817])

0.6932952924393724